In [ ]:
import json

with open("../data/raw/data.json", "r") as f:
    data = json.load(f)

type(data)

data.keys()

dict_keys(['metadata', 'timeseries'])

In [12]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd

records = data["timeseries"]
df = pd.DataFrame(records)
df.head()

,PRCP,SNWD,TMAX,SNOW,TMIN
1968-06-01,0.00,0.0,70.0,0.0,51.0
1968-06-02,0.00,0.0,76.0,0.0,57.0
1968-06-03,0.00,0.0,74.0,0.0,55.0
1968-06-04,0.02,0.0,78.0,0.0,54.0
1968-06-05,0.00,0.0,76.0,0.0,54.0


In [14]:
df.index = pd.to_datetime(df.index)
df.index.name = "date"
df = df.sort_index()

df.info()
df.describe()

<class 'pandas.DataFrame'>
DatetimeIndex: 21063 entries, 1968-06-01 to 2026-01-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PRCP    21057 non-null  float64
 1   SNWD    20974 non-null  float64
 2   TMAX    21044 non-null  float64
 3   SNOW    20991 non-null  float64
 4   TMIN    20990 non-null  float64
dtypes: float64(5)
memory usage: 987.3 KB


,PRCP,SNWD,TMAX,SNOW,TMIN
count,21057.000000,20974.000000,21044.000000,20991.000000,20990.000000
mean,0.135345,0.338324,63.446731,0.074637,43.394712
std,0.371700,1.567704,18.683399,0.640831,16.968509
min,0.000000,0.000000,0.000000,0.000000,-13.000000
25%,0.000000,0.000000,48.000000,0.000000,30.000000
50%,0.000000,0.000000,65.000000,0.000000,43.000000
75%,0.060000,0.000000,80.000000,0.000000,58.000000
max,7.960000,25.000000,105.000000,19.800000,80.000000


In [16]:
# create the lag windows
df["TMIN_next"] = df["TMIN"].shift(-1)
df["TMIN_lag1"] = df["TMIN"].shift(1)
df["TMIN_lag7"] = df["TMIN"].shift(7)
df["TMAX_lag1"] = df["TMAX"].shift(1)

# drops missing rows
df = df.dropna()

In [ ]:
df[[
    "TMIN",
    "TMIN_lag1",
    "TMIN_lag7",
    "TMAX_lag1",
    "TMIN_next"
]].head()

,TMIN,TMIN_lag1,TMIN_lag7,TMAX_lag1,TMIN_next
date,,,,,
1968-06-08,66.0,66.0,51.0,91.0,63.0
1968-06-09,63.0,66.0,57.0,92.0,67.0
1968-06-10,67.0,63.0,55.0,80.0,55.0
1968-06-11,55.0,67.0,54.0,88.0,55.0
1968-06-12,55.0,55.0,54.0,67.0,64.0


In [18]:
# naive predictor
df["pred_baseline"] = df["TMIN_lag1"]

# time-aware train/test split
train = df.loc[:'2015']
test = df.loc['2016':]

In [20]:
%pip install scikit-learn

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   -------------- ------------------------- 2.9/8.0 MB 14.0 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.0 MB 14.1 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 13.6 MB/s  0:00:00
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   -- ------------------------------------- 2.6/36.3 MB 13.6 MB/s eta 0:00:03
   ----- ---------------------------------- 4.7/36.3 MB 11.8 MB/s eta 0:00:03
   -------- ------------------------------- 7.3/36.3 MB 12.1 MB/s eta 0:00:03
   ----------- ---------------------------- 10.2/36.3 MB 12.6 MB/s eta 0:00:03
   -------------- ------------------------- 13.4/36.3 MB 13.0 MB/s eta 0:00:02
   ------------------ --------------------- 16.5/36.3 MB 13.2 MB/s eta 0:00:02
   --------------------- ------------------ 19.4/36.3 MB 13.4 MB/s eta 0:00:02
   ------------------------ --------------- 22.3/36.3 MB 13.5 MB/s eta 0:00:02
   -

In [19]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(
    test["TMIN_next"],
    test["pred_baseline"]
)

print("Baseline MAE:", mae)

ModuleNotFoundError: No module named 'sklearn'